In [1]:
# Import all libraries needed for the tutorial

# General syntax to import specific functions in a library: 
##from (library) import (specific library function)
from pandas import DataFrame, read_csv

# General syntax to import a library but no functions: 
##import (library) as (give the library a nickname/alias)
import matplotlib.pyplot as plt
import pandas as pd #this is how I usually import pandas
import sys #only needed to determine Python version number
import matplotlib #only needed to determine Matplotlib version number
import numpy as np

import matplotlib.pyplot as plt
import numpy as np
from sklearn import datasets, linear_model
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.cross_validation import cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PolynomialFeatures
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier

from sklearn import datasets
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression

from sklearn import model_selection

# Enable inline plotting
%matplotlib inline
print('Python version ' + sys.version)
print('Pandas version ' + pd.__version__)
print('Matplotlib version ' + matplotlib.__version__)  

Python version 2.7.13 (default, Dec 18 2016, 07:03:39) 
[GCC 4.2.1 Compatible Apple LLVM 8.0.0 (clang-800.0.42.1)]
Pandas version 0.22.0
Matplotlib version 1.5.3


/usr/local/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
def scaleData(inp, minimum, maximum):
    minMaxScaler = preprocessing.MinMaxScaler(copy=True, feature_range=(minimum,maximum))
    #inp = inp.reshape(inp.shape[0]*inp.shape[1])
    inp = minMaxScaler.fit_transform(inp)
    
    return inp

In [3]:
features = ['org','dst','leg','gap_in','gap_log',
            'booking_dayofweek','booking_month','booking_week','booking_dayofyear_in',
            'dep_dayofweek','dep_month','dep_week','dep_dayofyear_in',
            'dep_time_sc','pax']
feature_length = 15

def prepFeatures(df):
    maxPax  = 7
    numDaysInYear = 365
    numOfMonths = 12
    numOfDaysInWeek = 7
    numOfWeeksInYear = 52
    numClasses = 4
    epsilon = 0.0001

    df['org'] = df['origin'].astype('category').cat.codes
    df['dst'] = df['destination'].astype('category').cat.codes
    maxOrg = max(df['org'])+1
    maxDst = max(df['dst'])+1
    df['org'] = (df['org']+1)/maxOrg
    df['dst'] = (df['dst']+1)/maxDst
    df['leg'] = (df['org']+df['dst'])/(maxOrg+maxDst)
    df['gap'] = ((df['dep_date'] - df['booking_date'])/np.timedelta64(1, 'D')).astype(int)+1
    df['gap_in'] = df['gap'][np.abs(df['gap']-df['gap'].mean())<=(2*df['gap'].std())]
    df['gap_in'] = df['gap_in'].fillna(epsilon)
    df['gap_log'] = np.log(df['gap'])
    df['booking_dayofweek'] = (df['booking_date'].dt.dayofweek+1)/numOfDaysInWeek
    df['booking_dayofyear'] = (df['booking_date'].dt.dayofyear+1)/numDaysInYear
    df['booking_month'] = (df['booking_date'].dt.month+1)/numOfMonths
    df['booking_week'] = (df['booking_date'].dt.week+1)/numOfWeeksInYear
    df['booking_dayofyear_in'] = df['booking_dayofyear'][np.abs(df['booking_dayofyear']-df['booking_dayofyear'].mean())<=(3*df['booking_dayofyear'].std())]
    df['booking_dayofyear_in'] = df['booking_dayofyear_in'].fillna(epsilon)
    df['dep_dayofweek'] = (df['dep_date'].dt.dayofweek+1)/numOfDaysInWeek
    df['dep_dayofyear'] = (df['dep_date'].dt.dayofyear+1)/numDaysInYear
    df['dep_month'] = (df['dep_date'].dt.month+1)/numOfMonths
    df['dep_week'] = (df['dep_date'].dt.week+1)/numOfWeeksInYear
    df['dep_dayofyear_in'] = df['dep_dayofyear'][np.abs(df['dep_dayofyear']-df['dep_dayofyear'].mean())<=(3*df['dep_dayofyear'].std())]
    df['dep_dayofyear_in'] = df['dep_dayofyear_in'].fillna(epsilon)
    df['dep_time_sc'] = (((df['dep_time']%100).astype(int)/5)+1 + ((df['dep_time']/100).astype(int)+1)*12)/((24*12))
    df['pax'] = df['pax']/maxPax
    
    return df
    

In [4]:
Location = r'train.csv'
df = pd.read_csv(Location, parse_dates=['booking_date', 'dep_date'])

Location = r'test.csv'
test_df = pd.read_csv(Location, parse_dates=['booking_date', 'dep_date'])

df = prepFeatures(df)
df['fare_choice_sc'] = df.groupby(['fare_choice']).ngroup()+1
test_df = prepFeatures(test_df)

print((df.ix[56109]))
df.head()

id                                    56110
booking_date            2016-12-31 00:00:00
origin                                  BOM
destination                             CCU
dep_date                2017-06-18 00:00:00
dep_time                               1255
pax                                0.142857
fare_choice                            Deal
org                                0.285714
dst                                     0.6
leg                               0.0738095
gap                                     170
gap_in                               0.0001
gap_log                              5.1358
booking_dayofweek                  0.857143
booking_dayofyear                   1.00548
booking_month                       1.08333
booking_week                        1.01923
booking_dayofyear_in                 0.0001
dep_dayofweek                             1
dep_dayofyear                      0.465753
dep_month                          0.583333
dep_week                        

/usr/local/lib/python2.7/site-packages/ipykernel_launcher.py:11: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  # This is added back by InteractiveShellApp.init_path()


,id,booking_date,origin,destination,dep_date,dep_time,pax,fare_choice,org,dst,...,booking_month,booking_week,booking_dayofyear_in,dep_dayofweek,dep_dayofyear,dep_month,dep_week,dep_dayofyear_in,dep_time_sc,fare_choice_sc
0,1,2017-02-10,MAA,DEL,2017-06-01,630,0.285714,Deal,1.000000,0.8,...,0.25,0.134615,0.000100,0.571429,0.419178,0.583333,0.442308,0.419178,0.315972,2
1,2,2017-05-27,DEL,BOM,2017-06-01,1745,0.142857,Classic,0.571429,0.4,...,0.50,0.423077,0.405479,0.571429,0.419178,0.583333,0.442308,0.419178,0.784722,1
2,3,2017-05-30,BOM,DEL,2017-06-01,900,0.142857,Saver,0.285714,0.8,...,0.50,0.442308,0.413699,0.571429,0.419178,0.583333,0.442308,0.419178,0.420139,4
3,4,2017-05-30,BOM,BLR,2017-06-01,2130,0.285714,Classic,0.285714,0.2,...,0.50,0.442308,0.413699,0.571429,0.419178,0.583333,0.442308,0.419178,0.940972,1
4,5,2017-05-30,DEL,BOM,2017-06-01,1240,0.142857,Classic,0.571429,0.4,...,0.50,0.442308,0.413699,0.571429,0.419178,0.583333,0.442308,0.419178,0.572917,1


In [5]:
test_flightDataPd = test_df[features]
test_flightData = test_flightDataPd.as_matrix(columns=test_flightDataPd.columns[0:])

X_test = test_flightData[:,0:feature_length]
test_flightDataPd.head()

,org,dst,leg,gap_in,gap_log,booking_dayofweek,booking_month,booking_week,booking_dayofyear_in,dep_dayofweek,dep_month,dep_week,dep_dayofyear_in,dep_time_sc,pax
0,0.142857,0.8,0.078571,0.0001,4.174387,0.714286,0.416667,0.326923,0.306849,0.857143,0.583333,0.5,0.482192,0.784722,0.285714
1,0.285714,0.8,0.090476,32.0000,3.465736,0.428571,0.500000,0.423077,0.397260,0.857143,0.583333,0.5,0.482192,0.753472,0.285714
2,0.142857,0.8,0.078571,0.0001,4.304065,0.428571,0.416667,0.307692,0.282192,0.857143,0.583333,0.5,0.482192,0.336806,0.142857
3,0.142857,0.8,0.078571,4.0000,1.386294,0.428571,0.583333,0.500000,0.473973,0.857143,0.583333,0.5,0.482192,0.520833,0.285714
4,0.285714,0.6,0.073810,25.0000,3.218876,0.428571,0.500000,0.442308,0.416438,0.857143,0.583333,0.5,0.482192,0.840278,0.142857


In [6]:
features.append('fare_choice_sc')
flightDataPd = df[features]
flightData = flightDataPd.as_matrix(columns=flightDataPd.columns[0:])
X = flightData[:,0:feature_length]
y = flightData[:,feature_length]
flightDataPd.head()

,org,dst,leg,gap_in,gap_log,booking_dayofweek,booking_month,booking_week,booking_dayofyear_in,dep_dayofweek,dep_month,dep_week,dep_dayofyear_in,dep_time_sc,pax,fare_choice_sc
0,1.000000,0.8,0.150000,0.0001,4.718499,0.714286,0.25,0.134615,0.000100,0.571429,0.583333,0.442308,0.419178,0.315972,0.285714,2
1,0.571429,0.4,0.080952,6.0000,1.791759,0.857143,0.50,0.423077,0.405479,0.571429,0.583333,0.442308,0.419178,0.784722,0.142857,1
2,0.285714,0.8,0.090476,3.0000,1.098612,0.285714,0.50,0.442308,0.413699,0.571429,0.583333,0.442308,0.419178,0.420139,0.142857,4
3,0.285714,0.2,0.040476,3.0000,1.098612,0.285714,0.50,0.442308,0.413699,0.571429,0.583333,0.442308,0.419178,0.940972,0.285714,1
4,0.571429,0.4,0.080952,3.0000,1.098612,0.285714,0.50,0.442308,0.413699,0.571429,0.583333,0.442308,0.419178,0.572917,0.142857,1


In [7]:
print(X.shape)
poly = PolynomialFeatures(2)
X = poly.fit_transform(X)
print(X.shape)

(73830, 15)
(73830, 136)


In [ ]:
regr = LogisticRegression()
rfe = RFE(regr, 22)
rfe = rfe.fit(X, y )
print(rfe.support_)
print(rfe.ranking_)
X = X[rfe.support_]
print(X.shape)

In [ ]:
X_1, X_2, y_1, y_2 = train_test_split(X, y, test_size=0.3, random_state=42)

#LOGISTIC REGRESSION
#regr = linear_model.LogisticRegression()
# Train the model using the training sets
#regr.fit(data_X_train, data_y_train)

#Support Vector Classification
#regr = svm.SVC(C = 1.0, probability=True, kernel='rbf')
#regr.fit(data_X_train,data_y_train)

#RANDOM FOREST
regr = RandomForestClassifier(max_depth=7, random_state=1)
regr.fit(X_1,y_1)

In [ ]:
kfold = model_selection.KFold(n_splits=10, random_state=7)
scoring = 'accuracy'
results = model_selection.cross_val_score(regr, X, y, cv=kfold, scoring=scoring)
predictions = model_selection.cross_val_predict(regr, X, y, cv=kfold)
print("10-fold cross validation average accuracy: %.3f" % (results.mean()))

# The mean squared error
print("Mean absolute error: %.2f"
      % mean_absolute_error(y, predictions))
# Explained variance score: 1 is perfect prediction
print('Variance score: %.2f' % r2_score(y, predictions))

In [ ]:
X_test_prob = regr.predict_proba(X_test)
idx = 0
y_test_df = pd.DataFrame(X_test_prob)
y_test_df.insert(loc=idx, column='id', value=np.nan)
y_test_df['id'] = df_y_pred.index + 73831
y_test_df.head(20)

output_location = r'prediction.csv'
y_test_df.to_csv(output_location, index=False, header=['id', 'Classic', 'Deal', 'Flex', 'Saver'])
